In [1]:
import pandas as pd

# Load the pipe-separated data
df = pd.read_csv('../data/raw/insurance_data.txt', sep='|')


C:\Users\mike\AppData\Local\Temp\ipykernel_18260\4012117072.py:4: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/insurance_data.txt', sep='|')


In [19]:
# Import libraries

import pandas as pd

# Get some insights about the data
print(df.head())
print(df.info())
print(df.describe())
print(df.dtypes)
print(df.isna().sum())

   UnderwrittenCoverID  PolicyID TransactionMonth  IsVATRegistered  \
0               145249     12827       2015-03-01             True   
1               145249     12827       2015-05-01             True   
2               145249     12827       2015-07-01             True   
3               145255     12827       2015-05-01             True   
4               145255     12827       2015-07-01             True   

  Citizenship          LegalType Title Language                 Bank  \
0              Close Corporation    Mr  English  First National Bank   
1              Close Corporation    Mr  English  First National Bank   
2              Close Corporation    Mr  English  First National Bank   
3              Close Corporation    Mr  English  First National Bank   
4              Close Corporation    Mr  English  First National Bank   

       AccountType  ...                    ExcessSelected CoverCategory  \
0  Current account  ...             Mobility - Windscreen    Windscreen

In [ ]:
# Parse date columns

df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'])

df['VehicleIntroDate'] = pd.to_datetime(df['VehicleIntroDate'])

In [ ]:
#  Drop unnecessary columns with too many missing values
df.drop(columns=['NumberOfVehiclesInFleet', 'CrossBorder'], inplace=True)

#  impute some columns with missing value as unknown
for col in ['Bank', 'AccountType']:
    df[col].fillna('Unknown', inplace=True)

#  impute columns with Not specified
for col in ['MaritalStatus', 'Gender']:
    df[col].fillna('Not specified', inplace=True)

# Impute column with mode
df['CapitalOutstanding'].fillna(df['CapitalOutstanding'].mode()[0], inplace=True)


# Drop rows with missing values < 1%
df.dropna(subset=['mmcode', 'make', 'Model', 'VehicleIntroDate', 'bodytype', 'kilowatts','NumberOfDoors', 'cubiccapacity', 'Cylinders', 'VehicleType'], inplace=True)

# For CustomValueEstimate — if too risky, you may still drop
# df.drop(columns=['CustomValueEstimate'], inplace=True)


In [ ]:
# impute CustomValueEstimate column with median from the same corresponding make, model and registration year
 
# Group median by Make, Model, RegistrationYear
group_median = (
    df.groupby(['make', 'Model', 'RegistrationYear'])['CustomValueEstimate']
    .median()
    .reset_index()
    .rename(columns={'CustomValueEstimate': 'GroupMedianValue'})
)

# Merge back into main dataframe
df = df.merge(group_median, on=['make', 'Model', 'RegistrationYear'], how='left')

# Fill missing CustomValueEstimate using group median
df['CustomValueEstimate'] = df['CustomValueEstimate'].fillna(df['GroupMedianValue'])

# Drop the helper column
df.drop(columns=['GroupMedianValue'], inplace=True)

# Fill any remaining with global median
#df['CustomValueEstimate'].fillna(df['CustomValueEstimate'].median(), inplace=True)


In [ ]:
# Save cleaned data
df.to_csv('../data/processed/processed_insurance_data.csv', index=False)